# Orbita3d Controller API example

## Installation

The easiest way to install the Orbita3d Controller API is via the `maturin develop` command from the `orbita3d_c_api` folder.

Please refer to [their documentation](https://www.maturin.rs/) for more information.


To check the install, simply run the following cell:

In [1]:
from orbita3d import Orbita3dController

You can create a new controller using the from_config method.

Configuration examples can be found in [the orbita3d_controller pacakge](https://github.com/pollen-robotics/orbita3d_control/tree/develop/orbita3d_controller/config).

In [2]:
# Replace the path with the path to your config file
config = "/home/prod/dev_iwrist/orbita3d_control/orbita3d_controller/config/dxl_iwrist.yaml"

orbita3d_controller = Orbita3dController.from_config(config)

## Control your Orbita3d

### Enable/disable the torque

You can enable the torque using the `enable_torque` method. It also takes a `reset_target` argument that will reset the target position to the current position.

In [3]:
orbita3d_controller.enable_torque(reset_target=True)

You can now check that the torque was enabled by calling the `is_torque_on` method.

In [ ]:
orbita3d_controller.is_torque_on()

And now, you can disable the torque using the `disable_torque` method.

In [ ]:
orbita3d_controller.disable_torque()

Let's check that the torque is off.

In [ ]:
orbita3d_controller.is_torque_on()

### Read and set orientation

To read the current orientation, you can use the `get_current_orientation` method. It returns a quaternion (qx, qy, qz, qw).

_Note that you can use the [Rotation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html) module from the scipy library to convert the quaternion to a rotation matrix or euler angles._

In [4]:
q = orbita3d_controller.get_current_orientation()
q

(-0.08919770630298376,
 0.023292405485376273,
 -0.012345987509610643,
 0.9956650087400903)

In [5]:
from scipy.spatial.transform import Rotation

roll, pitch, yaw = Rotation.from_quat(q).as_euler('xyz', degrees=True)
roll, pitch, yaw

(-10.274911230350275, 2.532174599795125, -1.6485287036830465)

You can also set the target orientation using the `set_target_orientation` method. It takes a quaternion (qx, qy, qz, qw) as argument.

_Make sure to first enable the torque before setting the target orientation, otherwise your Orbita3d won't move._

In [ ]:
orbita3d_controller.enable_torque(reset_target=True)

In [7]:
# Let's move the robot to a new orientation
# By setting its yaw to 30, 60, 90 degrees and back

import time

for yaw in [30, 60, 90, 60, 30, 0]:
    target = Rotation.from_euler('xyz', [roll, pitch, yaw], degrees=True).as_quat()
    print(target)
    #orbita3d_controller.set_target_orientation(target)
    fb=orbita3d_controller.set_target_orientation_fb(target)
    time.sleep(1)
    #current = orbita3d_controller.get_current_orientation()
    print(f"Target: {target} - Current: {fb}")

[-0.0921689  -0.00191343  0.25962751  0.96129849]


TypeError: initializer for ctype 'double(*)[4]' must be a pointer to same type, not cdata 'double(*)[10]'

In [8]:
# Let's move the robot to a new orientation
# By setting its yaw to 30, 60, 90 degrees and back

import time

for yaw in [0]:
    target = Rotation.from_euler('xyz', [roll, pitch, yaw], degrees=True).as_quat()
    print(target)
    orbita3d_controller.set_target_orientation(target)
    #fb=orbita3d_controller.set_target_orientation_fb(target)
    time.sleep(1)
    #current = orbita3d_controller.get_current_orientation()
    print(f"Target: {target} - Current: {fb}")

[-0.08952355  0.02200683  0.00197856  0.99573958]


NameError: name 'fb' is not defined

We can also play a more complex trajectory and record the current orientation at each step.

In [ ]:
import numpy as np

dur = 10

pitch_sin_f = 0.25
pitch_sin_amp = 30

yaw_sin_f = 0.5
yaw_sin_amp = 60

data = []

t0 = time.time()
while time.time() - t0 < dur:
    t = time.time() - t0

    roll = 0.0
    pitch = pitch_sin_amp * np.sin(2 * np.pi * pitch_sin_f * t)
    yaw = yaw_sin_amp * np.sin(2 * np.pi * yaw_sin_f * t)

    target = Rotation.from_euler('xyz', [roll, pitch, yaw], degrees=True).as_quat()
    orbita3d_controller.set_target_orientation(target)
    time.sleep(0.01)

    current = orbita3d_controller.get_current_orientation()
    data.append(current)

Using matplotlib, let's have a look at our trajectory.

In [ ]:
import matplotlib.pyplot as plt

roll, pitch, yaw = Rotation.from_quat(data).as_euler('xyz', degrees=True).T

plt.figure()
plt.plot(roll, label="roll")
plt.plot(pitch, label="pitch")
plt.plot(yaw, label="yaw")
plt.legend()


You can also access the velocity and torque using respectively the `get_current_velocity` and `get_current_torque` methods.

### Extra parameters

There are some extra parameters that you can access. **In opposition to the other values, those parameters impact directly the raw motor on not the actuator!**

You can access:
* the velocity limit using the `get_raw_motors_velocity_limit` and `set_raw_motors_velocity_limit` methods
* the torque limit using the `get_raw_motors_torque_limit` and `set_raw_motors_torque_limit` methods
* the pid gains using the `get_raw_motors_pid_gains` and `set_raw_motors_pid_gains` methods

In [ ]:
orbita3d_controller.get_raw_motors_velocity_limit()

In [ ]:
orbita3d_controller.get_raw_motors_torque_limit()

In [ ]:
pids=orbita3d_controller.get_raw_motors_pid_gains()
pids[0][0]